In [3]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Import necessary libraries
import os, numpy as np, matplotlib.pyplot as plt, cv2, PIL, tensorflow as tf, io
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
from keras.models import load_model
from keras.layers import Dropout

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import os
from cv2 import imread, cvtColor, resize, COLOR_BGR2RGB
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import math

# Set the path to your dataset
data_path = '/content/drive/MyDrive/Colab Notebooks/money-dataset'

# Function to load and preprocess images
def load_images(path):
    images = []
    labels = []
    
    for folder in os.listdir(path):
        folder_path = os.path.join(path, folder)
        if os.path.isdir(folder_path):
            label = folder
            for file in os.listdir(folder_path):
                file_path = os.path.join(folder_path, file)
                if file.endswith('.jpg') or file.endswith('.jpeg') or file.endswith('.jfif'):
                    print(file_path)
                    img = imread(file_path)
                    img = cvtColor(img, COLOR_BGR2RGB)  # Convert BGR to RGB
                    img = resize(img, (128, 128))  # Resize the image to a consistent size
                    img = img.astype('float32') / 255.0  # Convert to float32 and normalize the pixel values
                    images.append(img)
                    labels.append(label)
    
    return np.array(images), np.array(labels)

# Load the images and labels
images, labels = load_images(data_path)

# Perform label encoding and one-hot encoding
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)
labels = to_categorical(labels)

# Save the label encoder classes
np.save('/content/drive/MyDrive/Colab Notebooks/label_encoder_classes.npy', label_encoder.classes_)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Normalize the pixel values of the test set
X_test = X_test.astype('float32') / 255.0

# Create an ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(
    rotation_range=10,  # Randomly rotate the images by 10 degrees
    zoom_range=0.1,  # Randomly zoom the images by 10%
    width_shift_range=0.1,  # Randomly shift the images horizontally by 10% of the total width
    height_shift_range=0.1,  # Randomly shift the images vertically by 10% of the total height
    horizontal_flip=True  # Randomly flip the images horizontally
)

# Build the CNN model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

# Compile the model
optimizer = Adam(lr=0.0001)  # Adjust the learning rate
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Define a learning rate reduction callback
reduce_lr = ReduceLROnPlateau(factor=0.1, patience=3)

# Calculate the steps per epoch
steps_per_epoch = math.ceil(len(X_train) / 32)

# Train the model with data augmentation
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    steps_per_epoch=steps_per_epoch,
    epochs=100,  # Increase the number of epochs
    verbose=1,
    validation_data=(X_test, y_test),
    callbacks=[reduce_lr]
)

# Evaluate the model
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# Save the model
model.save('/content/drive/MyDrive/Colab Notebooks/model.h5')

/content/drive/MyDrive/Colab Notebooks/money-dataset/50000 lama/7.jpg
/content/drive/MyDrive/Colab Notebooks/money-dataset/50000 lama/6.jfif
/content/drive/MyDrive/Colab Notebooks/money-dataset/50000 lama/4.jpg
/content/drive/MyDrive/Colab Notebooks/money-dataset/50000 lama/3.jfif
/content/drive/MyDrive/Colab Notebooks/money-dataset/50000 lama/5.jpg
/content/drive/MyDrive/Colab Notebooks/money-dataset/50000 lama/2.jpg
/content/drive/MyDrive/Colab Notebooks/money-dataset/50000 lama/1.jpeg
/content/drive/MyDrive/Colab Notebooks/money-dataset/50000 baru/9.jpg
/content/drive/MyDrive/Colab Notebooks/money-dataset/50000 baru/7.jpg
/content/drive/MyDrive/Colab Notebooks/money-dataset/50000 baru/6.jpg
/content/drive/MyDrive/Colab Notebooks/money-dataset/50000 baru/8.jpg
/content/drive/MyDrive/Colab Notebooks/money-dataset/50000 baru/5.jpg
/content/drive/MyDrive/Colab Notebooks/money-dataset/50000 baru/10.jpeg
/content/drive/MyDrive/Colab Notebooks/money-dataset/50000 baru/4.jpg
/content/drive/

Epoch 1/100
3/3 [==============================] - 5s 1s/step - loss: 7.2903 - accuracy: 0.0521 - val_loss: 2.6382 - val_accuracy: 0.0800 - lr: 0.0010
Epoch 2/100
3/3 [==============================] - 3s 1s/step - loss: 4.9314 - accuracy: 0.0729 - val_loss: 2.6392 - val_accuracy: 0.0800 - lr: 0.0010
Epoch 3/100
3/3 [==============================] - 5s 2s/step - loss: 2.7211 - accuracy: 0.1562 - val_loss: 2.6386 - val_accuracy: 0.0800 - lr: 0.0010
Epoch 4/100
3/3 [==============================] - 3s 1s/step - loss: 2.5990 - accuracy: 0.1458 - val_loss: 2.6378 - val_accuracy: 0.0800 - lr: 0.0010
Epoch 5/100
3/3 [==============================] - 3s 1s/step - loss: 2.5871 - accuracy: 0.0521 - val_loss: 2.6357 - val_accuracy: 0.0800 - lr: 0.0010
Epoch 6/100
3/3 [==============================] - 6s 2s/step - loss: 2.5734 - accuracy: 0.0729 - val_loss: 2.6268 - val_accuracy: 0.0800 - lr: 0.0010
Epoch 7/100
3/3 [==============================] - 3s 1s/step - loss: 2.4891 - accuracy: 0.197

In [4]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

In [5]:
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [ ]:
import numpy as np
import cv2
from keras.models import load_model

# Load the model
model = load_model('/content/drive/MyDrive/Colab Notebooks/model.h5')

# Load the label encoder classes
label_encoder_classes = np.load('/content/drive/MyDrive/Colab Notebooks/label_encoder_classes.npy')


# Start streaming video from webcam
video_stream()

# Label for video
label_html = 'Capture The Image'

# Initialize bounding box to empty
bbox = ''
count = 0 

while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # Convert JS response to OpenCV Image
    img = js_to_image(js_reply["img"])

    # Create transparent overlay for bounding box
    bbox_array = np.zeros([480, 640, 4], dtype=np.uint8)

    # Preprocess the image
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    img = cv2.resize(img, (128, 128))  # Resize the image to match the input size of the model
    img = img / 255.0  # Normalize the pixel values

    # Reshape the image to match the input shape of the model
    img = np.reshape(img, (1, 128, 128, 3))

    # Perform prediction using the loaded model
    prediction = model.predict(img, verbose=0)
    predicted_label_index = np.argmax(prediction)
    predicted_label = label_encoder_classes[predicted_label_index]
    confidence = prediction[0][predicted_label_index] * 100

    # Add the predicted label and confidence to the overlay image
    text = f"Label: {predicted_label}, Confidence: {confidence:.2f}"
    label_html = text

    # Convert overlay of bbox into bytes
    bbox_bytes = bbox_to_bytes(bbox_array)

    # Update bbox so the next frame gets a new overlay
    bbox = bbox_bytes

<IPython.core.display.Javascript object>